## Week 7: Antechamber Advance Tutorial

### Advance Tutorial 1: Setting up a DNA-Ligand System
#### https://ambermd.org/tutorials/advanced/tutorial1/

In [ ]:
# Develop charge parameters for SKE using this more advanced approach

# Charge derivation scheme is a crucial component of the force field parameterization process 
# In AMBER, the method used for charge derivation is called the Restrained Electrostatic Potential (RESP) method. 
# The RESP method involves fitting the partial charges to the electrostatic potential computed at a series of points surrounding the molecule. 
# This fitting is done under certain constraints to ensure that the charges are physically reasonable and that they reproduce the correct overall molecular dipole moment. 

In [ ]:
# There are several options for charge fitting in AMBER (original RESP implementation):
# 1) Antechamber: standalone ligand 
# 2) R.E.D.: complex systems because it is more reproducibility and is designed to automate multiple orientation fits 

### Part 1: Derive charges for the dye and linker

#### 1.1 Charge Derivation for the Dye

In [ ]:
# The charge fit consists of 3 stages:
# Stage 1: optimize the geometry
# Stage 2: generate an electrostatic potential
# Stage 3: run the RESP procedure

##### 1.1.1 Stage 1: optimize the geometry (optimize the capped dye)

In [ ]:
# Use B3LYP/6-31G* to optimize the geometry
# run optimization in a Gaussian QM package
# The dye has two minima that are in different conformations but equivalent in energy.
# We call these two conformations floF and floB

In [ ]:
$ module load amber/openmpi/intel/22.03

In [ ]:
# Get floB pre-prepared output files

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floB_opt.gout

In [ ]:
# Get floF pre-prepared output files

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floF_opt.gout

In [ ]:
# generate a pdb of the optimized floB structure for use later in the leap unit building
# Get floB pre-prepared output files

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floB_opt.pdb

##### 1.1.2 Stage 2: Calculate Electrostatic Potential

In [ ]:
# calculate an ESP for each of the two optimized geometries that can be read by the RESP program

In [ ]:
# Get floB pre-prepared output files

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floB_hf.gout

In [ ]:
# Get floF pre-prepared output files

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floF_hf.gout

##### 1.1.3 Convert the Gaussian ESP data into the RESP format

In [ ]:
# Make a esp.sh file
# xlf command not found
# Change xlf to gfortran

    """
#!/bin/bash
gfortran ./readit.f
grep "Atomic Center " $1 > a
grep "ESP Fit" $1 > b
grep "Fit    " $1 > c
./a.out 
rm -f a b c a.out readit.o
    """

In [ ]:
# How many atoms: atoms = 35
# How many ESP fit centers: ESP Fit centers = 79216

$ ./esp.sh floB_hf.gout

# enter natom,nesp:
# 35, 79216
# This should give an esp.dat file which we will rename ln5_esp.dat

##### 1.1.4 Generate the input files for RESP

In [ ]:
# generate the input files required for the RESP fit
# Make a floBF-resp.in file

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floBF-resp.in

    """
floBF-resp run #1
 &cntrl
 nmol=2,
 ihfree=1,
 qwt=0.0005,
 iqopt=2,
 /
    """
    
# nmol=2 tells RESP that this is a multiconformational fit over two 'molecules' (two conformations of the dye).
# ihfree=1 tells RESP we only want the weak restraint on the heavy atoms.
# qwt=0.0005 Strength of the restraint in A.U.
# iqopt=2 tells RESP to read in initial charges from a qin file in the form of Lagrange constraints. This is so we can fix the NME cap charges.

In [ ]:
$ cat floB_esp.dat floF_esp.dat > floBF_esp.dat

In [ ]:
# Make a qin file to provide the charges for the cap

$ wget https://ambermd.org/tutorials/advanced/tutorial1/files/floBF-resp.qin

In [ ]:
$ $AMBERHOME/bin/resp -O -i floBF-resp.in -o floBF-resp.out -p floBF-resp.pch -t floBF-resp.chg -q floBF-resp.qin -e floBF_esp.dat

# -O: Overwrite output files if they exist.
# -i floBF-resp.in: Input file for the RESP calculation.
# -o floBF-resp.out: Output file for the RESP calculation results.
# -p floBF-resp.pch: File to write the derived partial charges.
# -t floBF-resp.chg: File containing initial charges or charge constraints.
# -q floBF-resp.qin: File containing additional charge constraints (optional).
# -e floBF_esp.dat: File containing electrostatic potential data.

In [ ]:
# [qo210@cs441 Tutorial3]$ env | grep AMBER
# AMBER_LIB=/share/apps/amber/22.00/openmpi/intel/lib
# AMBER_INC=/share/apps/amber/22.00/openmpi/intel/include
# AMBER_ROOT=/share/apps/amber/22.00/openmpi/intel
# AMBERHOME=/share/apps/amber/22.00/openmpi/intel
# LMOD_FAMILY_AMBER=amber/openmpi/intel
# LMOD_FAMILY_AMBER_VERSION=22.00
# AMBER_HOME=/share/apps/amber/22.00/openmpi/intel